In [ ]:
import os
import logging

import gc

import torch
import torch.nn as nn
import pandas as pd
import numpy as np

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from pytorch_pretrained_bert.modeling import PreTrainedBertModel, BertModel
# from pytorch_pretrained_bert.tokenization import BertTokenizer
# from pytorch_pretrained_bert.optimization import BertAdam, warmup_linear, SCHEDULES
# from fastprogress import master_bar, progress_bar
from sklearn.model_selection import StratifiedShuffleSplit

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger("regressor")

FP16 = False
BATCH_SIZE = 32
SEED = 42
WARMUP_PROPORTION = 0.1
PYTORCH_PRETRAINED_BERT_CACHE = "/mnt/Intel/bert_tmp"
LOSS_SCALE = 0. # Loss scaling to improve fp16 numeric stability. Only used when fp16 set to True.
MAX_SEQ_LENGTH = 100

DATA_PATH = "./processed_data/en/ann_train_inl.csv"

In [ ]:
!pip install sklearn

     |████████████████████████████████| 7.3 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 30.9 MB 6.5 MB/s eta 0:00:01     |█████████████▌                  | 13.0 MB 3.8 MB/s eta 0:00:05
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=c2657882bbaa7fcf3051d8bec83fa788e9f603efa787675d05ee2fc17f89abef
  Stored in directory: /Users/hanhtran/Library/Caches/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
logger.info("device: {} n_gpu: {}, 16-bits training: {}".format(
    device, n_gpu, FP16))